In [13]:
%reset

import numpy as np
import os
import keras
import pydot
import pandas as pd

from keras import Model, optimizers, metrics
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import multi_gpu_model, plot_model, to_categorical
from keras.applications.vgg16 import VGG16

%matplotlib inline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [14]:
# Load Train Data(with regions)
train_data = np.load('train_data.npz')
print(train_data.files)

['cls_trn_img', 'cls_trn_lb', 'reg_trn_img', 'reg_trn_delta']


In [15]:
val_split = 0.2

reg_trn_img = train_data['reg_trn_img']
reg_trn_delta = train_data['reg_trn_delta']

In [16]:
import random

total = reg_trn_img.shape[0]
val_num = int(total * 0.2)


indexes = np.arange(total)
np.random.shuffle(indexes)
reg_trn_img = reg_trn_img[indexes]
reg_trn_delta = reg_trn_delta[indexes]

print(reg_trn_img.shape)
print(reg_trn_delta.shape)

train_img = reg_trn_img[val_num:]
train_delta = reg_trn_delta[val_num:]
val_img = reg_trn_img[:val_num]
val_delta = reg_trn_delta[:val_num]

print(train_img.shape)
print(train_delta.shape)
print(val_img.shape)
print(val_delta.shape)

(19070, 224, 224, 3)
(19070, 4)
(15256, 224, 224, 3)
(15256, 4)
(3814, 224, 224, 3)
(3814, 4)


In [22]:
# GPU Allocation Setting
    # import tensorflow as tf
    # from keras.backend.tensorflow_backend import set_session
    # config = tf.ConfigProto()
    # config.gpu_options.per_process_gpu_memory_fraction = 0.9
    # config.gpu_options.visible_device_list = '0'
    # set_session(tf.Session(config=config))

vggmodel = VGG16(weights='imagenet', include_top=True)

for layers in (vggmodel.layers)[:15]:
    layers.trainable = False
    
X = vggmodel.layers[-2].output
predictions = Dense(4, activation="linear")(X)

model_final = Model(input = vggmodel.input, output = predictions)

# Multi GPU
# model_final = multi_gpu_model(model_final, gpus=2)

opt = Adam(lr=0.0001)
model_final.compile(loss = losses.mean_squared_error, optimizer = opt, metrics=[metrics.mse])
model_final.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

/home/jhjung/tensorflow_1.14.0_env/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [23]:
# Model Save
model_json = model_final.to_json()
with open('network_model.json', 'w') as json_file:
    json_file.write(model_json)
# To Text File
with open('network_model.txt', 'w') as model_file:
    # Pass the file handle in as a lambda function to make it callable
    model_final.summary(print_fn=lambda x: model_file.write(x + '\n'))

# To Model Visualization
# plot_model(model_final, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [29]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
log = CSVLogger('log.csv', append=True, separator=';')
checkpoint = ModelCheckpoint(filepath='vgg16-airplane_{epoch:02d}_{val_loss:.4f}.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
callback_list = [log, checkpoint, early]

history = model_final.fit(reg_trn_img, reg_trn_delta, 
                    batch_size=256, epochs=1000,validation_split=0.2,
                    shuffle=True, verbose=2, callbacks=callback_list)
    
model_final.save_weights('trained_weight.h5')

Train on 15256 samples, validate on 3814 samples
Epoch 1/1000
 - 60s - loss: 0.1001 - mean_squared_error: 0.1001 - val_loss: 0.0853 - val_mean_squared_error: 0.0853

Epoch 00001: val_loss improved from inf to 0.08531, saving model to vgg16-airplane_01_0.0853.h5
Epoch 2/1000
 - 53s - loss: 0.0764 - mean_squared_error: 0.0764 - val_loss: 0.0675 - val_mean_squared_error: 0.0675

Epoch 00002: val_loss improved from 0.08531 to 0.06753, saving model to vgg16-airplane_02_0.0675.h5
Epoch 3/1000
 - 53s - loss: 0.0515 - mean_squared_error: 0.0515 - val_loss: 0.0546 - val_mean_squared_error: 0.0546

Epoch 00003: val_loss improved from 0.06753 to 0.05462, saving model to vgg16-airplane_03_0.0546.h5
Epoch 4/1000
 - 54s - loss: 0.0290 - mean_squared_error: 0.0290 - val_loss: 0.0420 - val_mean_squared_error: 0.0420

Epoch 00004: val_loss improved from 0.05462 to 0.04197, saving model to vgg16-airplane_04_0.0420.h5
Epoch 5/1000
 - 53s - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0364 - va


Epoch 00042: val_loss did not improve from 0.02498
Epoch 43/1000
 - 54s - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0256 - val_mean_squared_error: 0.0256

Epoch 00043: val_loss did not improve from 0.02498
Epoch 44/1000
 - 54s - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0246 - val_mean_squared_error: 0.0246

Epoch 00044: val_loss improved from 0.02498 to 0.02463, saving model to vgg16-airplane_44_0.0246.h5
Epoch 45/1000
 - 54s - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0247 - val_mean_squared_error: 0.0247

Epoch 00045: val_loss did not improve from 0.02463
Epoch 46/1000
 - 54s - loss: 9.3520e-04 - mean_squared_error: 9.3520e-04 - val_loss: 0.0248 - val_mean_squared_error: 0.0248

Epoch 00046: val_loss did not improve from 0.02463
Epoch 47/1000
 - 55s - loss: 7.9620e-04 - mean_squared_error: 7.9620e-04 - val_loss: 0.0245 - val_mean_squared_error: 0.0245

Epoch 00047: val_loss improved from 0.02463 to 0.02454, saving model to vgg16-airplane_47

 - 54s - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0234 - val_mean_squared_error: 0.0234

Epoch 00086: val_loss did not improve from 0.02293
Epoch 87/1000
 - 54s - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0228 - val_mean_squared_error: 0.0228

Epoch 00087: val_loss improved from 0.02293 to 0.02278, saving model to vgg16-airplane_87_0.0228.h5
Epoch 88/1000
 - 54s - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0229 - val_mean_squared_error: 0.0229

Epoch 00088: val_loss did not improve from 0.02278
Epoch 89/1000
 - 54s - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0225 - val_mean_squared_error: 0.0225

Epoch 00089: val_loss improved from 0.02278 to 0.02248, saving model to vgg16-airplane_89_0.0225.h5
Epoch 90/1000
 - 55s - loss: 9.0269e-04 - mean_squared_error: 9.0269e-04 - val_loss: 0.0227 - val_mean_squared_error: 0.0227

Epoch 00090: val_loss did not improve from 0.02248
Epoch 91/1000
 - 54s - loss: 8.3008e-04 - mean_squared_error: 8

 - 55s - loss: 5.4421e-04 - mean_squared_error: 5.4421e-04 - val_loss: 0.0210 - val_mean_squared_error: 0.0210

Epoch 00130: val_loss improved from 0.02119 to 0.02101, saving model to vgg16-airplane_130_0.0210.h5
Epoch 131/1000
 - 54s - loss: 4.4915e-04 - mean_squared_error: 4.4915e-04 - val_loss: 0.0212 - val_mean_squared_error: 0.0212

Epoch 00131: val_loss did not improve from 0.02101
Epoch 132/1000
 - 53s - loss: 4.4184e-04 - mean_squared_error: 4.4184e-04 - val_loss: 0.0213 - val_mean_squared_error: 0.0213

Epoch 00132: val_loss did not improve from 0.02101
Epoch 133/1000
 - 54s - loss: 4.4295e-04 - mean_squared_error: 4.4295e-04 - val_loss: 0.0209 - val_mean_squared_error: 0.0209

Epoch 00133: val_loss improved from 0.02101 to 0.02090, saving model to vgg16-airplane_133_0.0209.h5
Epoch 134/1000
 - 53s - loss: 4.2908e-04 - mean_squared_error: 4.2908e-04 - val_loss: 0.0209 - val_mean_squared_error: 0.0209

Epoch 00134: val_loss did not improve from 0.02090
Epoch 135/1000
 - 54s - l

 - 54s - loss: 3.6001e-04 - mean_squared_error: 3.6001e-04 - val_loss: 0.0201 - val_mean_squared_error: 0.0201

Epoch 00173: val_loss did not improve from 0.01955
Epoch 174/1000
 - 54s - loss: 4.1448e-04 - mean_squared_error: 4.1448e-04 - val_loss: 0.0197 - val_mean_squared_error: 0.0197

Epoch 00174: val_loss did not improve from 0.01955
Epoch 175/1000
 - 54s - loss: 5.2246e-04 - mean_squared_error: 5.2246e-04 - val_loss: 0.0205 - val_mean_squared_error: 0.0205

Epoch 00175: val_loss did not improve from 0.01955
Epoch 176/1000
 - 54s - loss: 6.9153e-04 - mean_squared_error: 6.9153e-04 - val_loss: 0.0196 - val_mean_squared_error: 0.0196

Epoch 00176: val_loss did not improve from 0.01955
Epoch 177/1000
 - 54s - loss: 8.1687e-04 - mean_squared_error: 8.1687e-04 - val_loss: 0.0202 - val_mean_squared_error: 0.0202

Epoch 00177: val_loss did not improve from 0.01955
Epoch 178/1000
 - 54s - loss: 9.1807e-04 - mean_squared_error: 9.1807e-04 - val_loss: 0.0204 - val_mean_squared_error: 0.0204

...


Epoch 00261: val_loss did not improve from 0.01709
Epoch 262/1000
 - 55s - loss: 2.7895e-04 - mean_squared_error: 2.7895e-04 - val_loss: 0.0177 - val_mean_squared_error: 0.0177

Epoch 00262: val_loss did not improve from 0.01709
Epoch 263/1000
 - 54s - loss: 4.3458e-04 - mean_squared_error: 4.3458e-04 - val_loss: 0.0181 - val_mean_squared_error: 0.0181

Epoch 00263: val_loss did not improve from 0.01709
Epoch 264/1000
 - 53s - loss: 5.8900e-04 - mean_squared_error: 5.8900e-04 - val_loss: 0.0184 - val_mean_squared_error: 0.0184

Epoch 00264: val_loss did not improve from 0.01709
Epoch 265/1000
 - 54s - loss: 8.7794e-04 - mean_squared_error: 8.7794e-04 - val_loss: 0.0197 - val_mean_squared_error: 0.0197

Epoch 00265: val_loss did not improve from 0.01709
Epoch 266/1000
 - 54s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0193 - val_mean_squared_error: 0.0193

Epoch 00266: val_loss did not improve from 0.01709
Epoch 267/1000
 - 54s - loss: 0.0020 - mean_squared_error: 0.0020 


Epoch 00306: val_loss did not improve from 0.01617
Epoch 307/1000
 - 53s - loss: 1.6818e-04 - mean_squared_error: 1.6818e-04 - val_loss: 0.0164 - val_mean_squared_error: 0.0164

Epoch 00307: val_loss did not improve from 0.01617
Epoch 308/1000
 - 53s - loss: 1.7060e-04 - mean_squared_error: 1.7060e-04 - val_loss: 0.0168 - val_mean_squared_error: 0.0168

Epoch 00308: val_loss did not improve from 0.01617
Epoch 309/1000
 - 53s - loss: 1.8394e-04 - mean_squared_error: 1.8394e-04 - val_loss: 0.0165 - val_mean_squared_error: 0.0165

Epoch 00309: val_loss did not improve from 0.01617
Epoch 310/1000
 - 54s - loss: 2.0227e-04 - mean_squared_error: 2.0227e-04 - val_loss: 0.0169 - val_mean_squared_error: 0.0169

Epoch 00310: val_loss did not improve from 0.01617
Epoch 311/1000
 - 54s - loss: 2.2529e-04 - mean_squared_error: 2.2529e-04 - val_loss: 0.0166 - val_mean_squared_error: 0.0166

Epoch 00311: val_loss did not improve from 0.01617
Epoch 312/1000
 - 54s - loss: 2.3163e-04 - mean_squared_er


Epoch 00352: val_loss did not improve from 0.01590
Epoch 353/1000
 - 54s - loss: 1.0703e-04 - mean_squared_error: 1.0703e-04 - val_loss: 0.0161 - val_mean_squared_error: 0.0161

Epoch 00353: val_loss did not improve from 0.01590
Epoch 354/1000
 - 54s - loss: 9.5911e-05 - mean_squared_error: 9.5911e-05 - val_loss: 0.0162 - val_mean_squared_error: 0.0162

Epoch 00354: val_loss did not improve from 0.01590
Epoch 355/1000
 - 53s - loss: 8.8159e-05 - mean_squared_error: 8.8159e-05 - val_loss: 0.0165 - val_mean_squared_error: 0.0165

Epoch 00355: val_loss did not improve from 0.01590
Epoch 356/1000
 - 54s - loss: 9.4270e-05 - mean_squared_error: 9.4270e-05 - val_loss: 0.0162 - val_mean_squared_error: 0.0162

Epoch 00356: val_loss did not improve from 0.01590
Epoch 357/1000
 - 53s - loss: 9.4219e-05 - mean_squared_error: 9.4219e-05 - val_loss: 0.0159 - val_mean_squared_error: 0.0159

Epoch 00357: val_loss did not improve from 0.01590
Epoch 358/1000
 - 54s - loss: 9.5433e-05 - mean_squared_er

 - 53s - loss: 7.3635e-05 - mean_squared_error: 7.3635e-05 - val_loss: 0.0164 - val_mean_squared_error: 0.0164

Epoch 00399: val_loss did not improve from 0.01590
Epoch 400/1000
 - 54s - loss: 7.0380e-05 - mean_squared_error: 7.0380e-05 - val_loss: 0.0162 - val_mean_squared_error: 0.0162

Epoch 00400: val_loss did not improve from 0.01590
Epoch 401/1000
 - 54s - loss: 5.8780e-05 - mean_squared_error: 5.8780e-05 - val_loss: 0.0163 - val_mean_squared_error: 0.0163

Epoch 00401: val_loss did not improve from 0.01590
Epoch 402/1000
 - 54s - loss: 5.3351e-05 - mean_squared_error: 5.3351e-05 - val_loss: 0.0163 - val_mean_squared_error: 0.0163

Epoch 00402: val_loss did not improve from 0.01590
Epoch 403/1000
 - 54s - loss: 5.8433e-05 - mean_squared_error: 5.8433e-05 - val_loss: 0.0163 - val_mean_squared_error: 0.0163

Epoch 00403: val_loss did not improve from 0.01590
Epoch 404/1000
 - 55s - loss: 5.5065e-05 - mean_squared_error: 5.5065e-05 - val_loss: 0.0165 - val_mean_squared_error: 0.0165

AttributeError: 'Model' object has no attribute 'save_weight'